In [6]:
using Plots
using Random

f(x) = (2-x)^2

# initial_pop_size = 20        
n_generations = 4   
x_range = (-5.0, 5.0) 
tournament_size = 3
max_age = 2  

mutable struct Indiv
    value::Float64
    age::Int
    new_this_generation::Bool  
end

function linear_crossover(parent1::Float64, parent2::Float64)
    alpha = rand()
    return alpha * parent1 + (1 - alpha) * parent2
end

function tournament_selection(population, fitness_values, tournament_size)
    indices = rand(1:length(population), tournament_size)
    winner_idx = indices[argmin([fitness_values[i] for i in indices])]
    return population[winner_idx]
end

population = [Indiv(1, 0, false), Indiv(2, 0, false), Indiv(3, 0, false), Indiv(4, 0, false), Indiv(5, 0, false)]

best_x_per_gen = Float64[]
best_y_per_gen = Float64[]
pop_size_per_gen = Int[]
all_populations = []
push!(all_populations, deepcopy(population))

for generation in 1:n_generations
    for ind in population
        ind.age += 1
        ind.new_this_generation = false
    end
    
    values = [f(ind.value) for ind in population]
    best_index = argmin(values)
    push!(best_x_per_gen, population[best_index].value)
    push!(best_y_per_gen, values[best_index])
    push!(pop_size_per_gen, length(population))
    
    filter!(ind -> ind.age < max_age, population)
    
    new_Indivs = []
    for _ in 1:5  
        parent1 = tournament_selection(population, values, tournament_size)
        parent2 = tournament_selection(population, values, tournament_size)
        
        if rand() < 0.7
            child_value = linear_crossover(parent1.value, parent2.value)
        else
            child_value = rand(Bool) ? parent1.value : parent2.value
        end
        
        if rand() < 0.1
            child_value += randn() * 0.5
            child_value = clamp(child_value, x_range[1], x_range[2])
        end
        
        push!(new_Indivs, Indiv(child_value, 0, true))
    end
    append!(population, new_Indivs)
    push!(all_populations, deepcopy(population))
end

xs = range(-5, 5, length=400)
ys = [f(x) for x in xs]
anim = @animate for i in 1:n_generations
    current_pop = all_populations[i]
    p = plot(xs, ys, label="f(x)", title="Generation $(i)", xlabel="x", ylabel="f(x)", legend=:topright, lw=2, ylims=(-10, 50))
    
    # Отрисовка популяции
    for ind in current_pop
        color = ind.new_this_generation ? :red : :grey
        alpha = ind.new_this_generation ? 1.0 : (max_age - ind.age)/max_age
        scatter!([ind.value], [f(ind.value)], color=color, alpha=alpha, markersize=6, label="")
    end
    
    # Лучшая особь
    scatter!([best_x_per_gen[i]], [best_y_per_gen[i]], color=:blue, markersize=8, markershape=:star5, label="Best")
    
    sleep(0.01)
end

gif(anim, "ga_linear_crossover.gif", fps=10)
println("Minimum: x = $(best_x_per_gen[end]), f(x) = $(best_y_per_gen[end])")
println("Final population size: $(length(population))")

Minimum: x = 2.0819947060302355, f(x) = 0.00672313181698474
Final population size: 10


┌ Info: Saved animation to /Users/d.okutin/ga_linear_crossover.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/FFuQi/src/animation.jl:156
